# MVP: skips NA'

The above will be fixed in iteration 2.

## Q. Can we predict the CSF and CYC spindles?

This is where the data lives (correct to an online link):

In [1]:
!ls /Volumes/Magdalena_NEW1/A_SPINDLE_ML_PROJECT/AWG_TORRENT/A_PAPER2_PIPELINE/Complete\ Dataset/Dataset

DefaultDB.db                 MyExpt_FilteredSpindles.csv
DefaultDB_MyExpt.properties  MyExpt_Image.csv
MyExpt_Chromatin.csv         MyExpt_PreSpindles.csv
MyExpt_Experiment.csv        MyExpt_Spindles.csv
MyExpt_FilteredChromatin.csv


In [2]:
!pwd

/Volumes/Seagate_RED/A_Heald_Lab/A_My_papers/A_TECH_PAPER/Super-Spindle-DataSet


Import necessary modules

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.mpl_style','default')
%matplotlib inline

import re

List versions

In [4]:
print 'pandas: ',pd.__version__
print 'numpy: ', np.__version__


pandas:  0.15.1
numpy:  1.9.1


In [5]:
df1=pd.read_csv('/Volumes/Magdalena_NEW1/A_SPINDLE_ML_PROJECT/AWG_TORRENT/A_PAPER2_PIPELINE/m')

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24321 entries, 0 to 24320
Columns: 203 entries, ImageNumber to Parent_Spindles
dtypes: float64(189), int64(10), object(4)
memory usage: 37.9+ MB


In [7]:
df1.head(1)

,ImageNumber,ObjectNumber,Metadata_Experiment,Metadata_Experimenter,Metadata_FileLocation,Metadata_Frame,Metadata_Series,Metadata_Set,Metadata_Treatment,Metadata_Type,...,Mean_FilteredChromatin_Location_CenterMassIntensity_Y_Rhodamine,Mean_FilteredChromatin_Location_Center_X,Mean_FilteredChromatin_Location_Center_Y,Mean_FilteredChromatin_Location_MaxIntensity_X_DNA,Mean_FilteredChromatin_Location_MaxIntensity_X_Rhodamine,Mean_FilteredChromatin_Location_MaxIntensity_Y_DNA,Mean_FilteredChromatin_Location_MaxIntensity_Y_Rhodamine,Mean_FilteredChromatin_Number_Object_Number,Number_Object_Number,Parent_Spindles
0,1,1,Exp00,AWG,NaN,0,0,stage10.5A_40x_second scope,105,STG,...,532.677614,737.334284,533.278871,732,764,542,535,1,1,1


In [8]:
df1.columns

Index([u'ImageNumber', u'ObjectNumber', u'Metadata_Experiment', u'Metadata_Experimenter', u'Metadata_FileLocation', u'Metadata_Frame', u'Metadata_Series', u'Metadata_Set', u'Metadata_Treatment', u'Metadata_Type', u'Metadata_cvsp', u'AreaShape_Area', u'AreaShape_Center_X', u'AreaShape_Center_Y', u'AreaShape_Compactness', u'AreaShape_Eccentricity', u'AreaShape_EulerNumber', u'AreaShape_Extent', u'AreaShape_FormFactor', u'AreaShape_MajorAxisLength', u'AreaShape_MaxFeretDiameter', u'AreaShape_MaximumRadius', u'AreaShape_MeanRadius', u'AreaShape_MedianRadius', u'AreaShape_MinFeretDiameter', u'AreaShape_MinorAxisLength', u'AreaShape_Orientation', u'AreaShape_Perimeter', u'AreaShape_Solidity', u'AreaShape_Zernike_0_0', u'AreaShape_Zernike_1_1', u'AreaShape_Zernike_2_0', u'AreaShape_Zernike_2_2', u'AreaShape_Zernike_3_1', u'AreaShape_Zernike_3_3', u'AreaShape_Zernike_4_0', u'AreaShape_Zernike_4_2', u'AreaShape_Zernike_4_4', u'AreaShape_Zernike_5_1', u'AreaShape_Zernike_5_3', u'AreaShape_Zernik

Data comes from several people. How many?

In [9]:
np.unique(df1.Metadata_Experimenter)

array(['AWG', 'KHR', 'MAS', 'MEC'], dtype=object)

Data comes from 4 people. What is the individual split?

In [10]:
grouped_by_person=df1.groupby('Metadata_Experimenter')
grouped_by_person.count().iloc[:,0]

Metadata_Experimenter
AWG                      10260
KHR                       8190
MAS                       3250
MEC                       2621
Name: ImageNumber, dtype: int64

We will need to subsample to predict the person. Which is the column to predict?

In [11]:
np.unique(df1.Metadata_Treatment)

array([  0,   1,   2,   3,   4,   5,   6,   8,  80,  84,  86,  87, 105,
       130, 131, 150, 160, 171, 200, 201, 203, 205, 302, 349, 420, 501,
       503, 505, 512, 800, 850, 875, 890, 895, 899])

In [12]:
np.unique(df1.Metadata_Type)

array(['BDS', 'CSF', 'CYC', 'STG'], dtype=object)

The first approximation seeks to distinguish CSF from CYC spindles. Chromatin is not used. Again, in this first project attempt, NA's will simmply be dropped. This is not the ultimate way to approach the project. We will use more sophisicated ways to deal with NA's in the subsequent versions.

Let's grab the metadata and drop everything apart from the target column (Metadata_Type).

In [13]:
meta_cols=[]
for col in df1.columns:
    try:
        meta_cols.append(re.search('(^Meta.+)',col).group(1))
    except AttributeError:
        continue

In [14]:
meta_cols.insert(0,df1.columns[0])

In [15]:
meta_cols.insert(1,df1.columns[1])

In [16]:
len(df1.columns)

203

In [17]:
len(meta_cols)

11

In [18]:
df1.columns[0:11]

Index([u'ImageNumber', u'ObjectNumber', u'Metadata_Experiment', u'Metadata_Experimenter', u'Metadata_FileLocation', u'Metadata_Frame', u'Metadata_Series', u'Metadata_Set', u'Metadata_Treatment', u'Metadata_Type', u'Metadata_cvsp'], dtype='object')

The columns above are Metadata columns that can be used as categories. 

### Pre-processing

#### 1. Missing data 

In [27]:
df1_numericals=df1[list(set(meta_cols).symmetric_difference(df1.columns))]

In [28]:
np.sum(np.sum(df1_numericals.isnull()))

48642

In [29]:
100*np.sum(np.sum(df1_numericals.isnull()))/(float(df1.shape[0])*df1.shape[1])

0.98522167487684731

There are 48642 numerical values missing in the data set (only 0.98% of the complete data set).

In [22]:
?any()


In [32]:
for col in df1_numericals.columns:
    if any(df1_numericals[col].isnull()):
        print col

Mean_FilteredChromatin_Distance_Minimum_FilteredSpindles
Mean_FilteredChromatin_Distance_Centroid_FilteredSpindles


In [35]:
all(df1_numericals.Mean_FilteredChromatin_Distance_Minimum_FilteredSpindles.isnull())

True

In [36]:
all(df1_numericals.Mean_FilteredChromatin_Distance_Minimum_FilteredSpindles.isnull())

True

In [39]:
np.sum(df1_numericals.Mean_FilteredChromatin_Distance_Minimum_FilteredSpindles.isnull())+np.sum(df1_numericals.Mean_FilteredChromatin_Distance_Minimum_FilteredSpindles.isnull())

48642

In [41]:
#df1.Mean_FilteredChromatin_Distance_Minimum_FilteredSpindles
#df1.Mean_FilteredChromatin_Distance_Centroid_FilteredSpindles

Looks like the two columns account for all missinga values. Everything else is simply ready to use.

####2. Centering and scaling